In [8]:
import re
import sys
phone_pattern ='(\d{3}[-\.\s/]??\d{3}[-\.\s/]??\d{4}|\(\d{3}\)\s*\d{3}[-\.\s/]??\d{4})'

# compiling the reg_ex would save sime time!
ph_reg = re.compile(phone_pattern)
age_reg = r'\d{1,3}';
# TODO for age: age_reg

In [9]:
'''
From perl
# Age indicators that follow ages
@age_indicators_suff = ("year old", "y\. o\.", "y\.o\.", "yo", "y", "years old", "year-old", "-year-old", "years-old", "-years-old", "years of age", "yrs of age");

# Age indicators that precede ages
@age_indicators_pre = ("age", "he is", "she is", "patient is");

# Digits, used in identifying ages
@digits = ("one","two","three","four","five","six","seven","eight","nine", "");


# Function: age()
# Checks for ages that are >=90 (assuming that no one is over 125 years old, just as a sanity check)
# When ages are spelled-out, assumes that the number will not be over hundred, i.e. highest spelled-out age="hundred"

sub age {
    $text = $_[0];   
    if ($agefilter =~ /y/) {
	
	foreach $i (@age_indicators_suff) {
	    if (($text =~ /\b(ninety)[\s\-]+ *$i\b/ig) || ($text =~ /\b(hundred)[\s\-]+ *$i\b/ig)) {
		my $age = $1;
		my $st = length($`);
		my $key = "$st-".((length $age) + $st);
		addType ($key, "Age over 90");
	    }
	    while ($text =~ /\b(([A-Za-z]+)([\s \-])([A-Za-z]+)) ? *$i\b/ig) {   		  
		foreach $j (@digits) {
		    $first = $2;
		    $second = $4;
		    my $age1 = $1;
		    my $st1 = length($`);
		    my $end1 = (length $age1) + $st1;
		    my $key1 = "$st1-$end1";
		    $st2 = $st1+length($2)+length($3);
		    $end2 = $st2+(length($second));
		    my $key2 = "$st2-$end2";
		    if ((($first=~/\bninety\b/ig) || ($first=~/\bhundred\b/ig)) && (($second=~/\b$digits\b/ig))) {
			addType ($key1, "Age over 90");	    
		    }
		    else {
			if (!(($first=~/\bninety\b/ig) || ($first=~/\bhundred\b/ig))) {
			    if (($second=~/\bninety\b/ig) || ($second=~/\bhundred\b/ig)) {
				addType ($key2, "Age over 90");
			    }
			}
		    }
		}
	    }
	    	    
	    while ($text =~ /\b(\d+) *$i/ig) {    
		my $age = $1;
		my $st = length($`);
		my $key = "$st-".((length $age) + $st);
		if (($age >= 90) && ($age <= 125)) {
		    addType ($key, "Age over 90");
		}
	    }
	}
	
	foreach $i (@age_indicators_pre) {
	    while ($text =~ /\b($i + *)(([A-Za-z]+)([\s \-])([A-Za-z]+))\b/ig) {   		  
		foreach $j (@digits) {
		    $first = $3;
		    $second = $5;
		    my $age1 = $2;
		    my $st1 = length($`)+length($1);
		    my $end1 = (length $age1) + $st1;
		    my $key1 = "$st1-$end1";
		    $st2 = $st1;
		    $end2 = $st2+length($first);
		    my $key2 = "$st2-$end2";
		    if ((($first=~/\bninety\b/ig) || ($first=~/\bhundred\b/ig)) && (($second=~/\b$digits\b/ig) || (length($second)))) {
			addType ($key1, "Age over 90");	    
		    }
		    else {
			if (!(($first=~/\bninety\b/ig) || ($first=~/\bhundred\b/ig))) {
			    if (($second=~/\bninety\b/ig) || ($second=~/\bhundred\b/ig)) {
				addType ($key2, "Age over 90");
			    }
			}
		    }
		}
	    }
	    
	    
	    while ($text =~ /\b($i + *)(\d+)\b/ig) {    
		my $age = $2;
		my $st = length($`)+length($1);
		my $key = "$st-".((length $age) + $st);
		if (($age >= 90) && ($age <= 125)) {
		    addType ($key, "Age over 90");
		}
	    }
	}
    }
}
# End of function age()

'''



def check_for_age(patient,note,chunk, output_handle):
    """
    Inputs:
        - patient: Patient Number, will be printed in each occurance of personal information found
        - note: Note Number, will be printed in each occurance of personal information found
        - chunk: one whole record of a patient
        - output_handle: an opened file handle. The results will be written to this file.
            to avoid the time intensive operation of opening and closing the file multiple times
            during the de-identification process, the file is opened beforehand and the handle is passed
            to this function. 
    Logic:
        Search the entire chunk for age occurances. Find the location of these occurances 
        relative to the start of the chunk, and output these to the output_handle file. 
        If there are no occurances, only output Patient X Note Y (X and Y are passed in as inputs) in one line.
        Use the precompiled regular expression to find age.
    """
    # The perl code handles texts a bit differently, 
    # we found that adding this offset to start and end positions would produce the same results
    offset = 27

    # For each new note, the first line should be Patient X Note Y and then all the personal information positions
    output_handle.write('Patient {}\tNote {}\n'.format(patient,note))

    # search the whole chunk, and find every position that matches the regular expression
    # for each one write the results: "Start Start END"
    # Also for debugging purposes display on the screen (and don't write to file) 
    # the start, end and the actual personal information that we found
    for match in age_reg.finditer(chunk):
                
            # debug print, 'end=" "' stops print() from adding a new line
            print(patient, note,end=' ')
            print((match.start()-offset),match.end()-offset, match.group())
                
            # create the string that we want to write to file ('start start end')    
            result = str(match.start()-offset) + ' ' + str(match.start()-offset) +' '+ str(match.end()-offset) 
            
            # write the result to one line of output
            output_handle.write(result+'\n')

In [10]:
def check_for_phone(patient,note,chunk, output_handle):
    """
    Inputs:
        - patient: Patient Number, will be printed in each occurance of personal information found
        - note: Note Number, will be printed in each occurance of personal information found
        - chunk: one whole record of a patient
        - output_handle: an opened file handle. The results will be written to this file.
            to avoid the time intensive operation of opening and closing the file multiple times
            during the de-identification process, the file is opened beforehand and the handle is passed
            to this function. 
    Logic:
        Search the entire chunk for phone number occurances. Find the location of these occurances 
        relative to the start of the chunk, and output these to the output_handle file. 
        If there are no occurances, only output Patient X Note Y (X and Y are passed in as inputs) in one line.
        Use the precompiled regular expression to find phones.
    """
    # The perl code handles texts a bit differently, 
    # we found that adding this offset to start and end positions would produce the same results
    offset = 27

    # For each new note, the first line should be Patient X Note Y and then all the personal information positions
    output_handle.write('Patient {}\tNote {}\n'.format(patient,note))

    # search the whole chunk, and find every position that matches the regular expression
    # for each one write the results: "Start Start END"
    # Also for debugging purposes display on the screen (and don't write to file) 
    # the start, end and the actual personal information that we found
    for match in ph_reg.finditer(chunk):
                
            # debug print, 'end=" "' stops print() from adding a new line
            print(patient, note,end=' ')
            print((match.start()-offset),match.end()-offset, match.group())
                
            # create the string that we want to write to file ('start start end')    
            result = str(match.start()-offset) + ' ' + str(match.start()-offset) +' '+ str(match.end()-offset) 
            
            # write the result to one line of output
            output_handle.write(result+'\n')


In [11]:
def deid_phone(text_path= 'id.text', output_path = 'phone.phi'):
    """
    Inputs: 
        - text_path: path to the file containing patient records
        - output_path: path to the output file.
    
    Outputs:
        for each patient note, the output file will start by a line declaring the note in the format of:
            Patient X Note Y
        then for each phone number found, it will have another line in the format of:
            start start end
        where the start is the start position of the detected phone number string, and end is the detected
        end position of the string both relative to the start of the patient note.
        If there is no phone number detected in the patient note, only the first line (Patient X Note Y) is printed
        to the output
    Screen Display:
        For each phone number detected, the following information will be displayed on the screen for debugging purposes 
        (these will not be written to the output file):
            start end phone_number
        where `start` is the start position of the detected phone number string, and `end` is the detected end position of the string
        both relative to the start of patient note.
    
    """
    # start of each note has the patter: START_OF_RECORD=PATIENT||||NOTE||||
    # where PATIENT is the patient number and NOTE is the note number.
    start_of_record_pattern = '^start_of_record=(\d+)\|\|\|\|(\d+)\|\|\|\|$'

    # end of each note has the patter: ||||END_OF_RECORD
    end_of_record_pattern = '\|\|\|\|END_OF_RECORD$'

    # open the output file just once to save time on the time intensive IO
    with open(output_path,'w+') as output_file:
        with open(text_path) as text:
            # initilize an empty chunk. Go through the input file line by line
            # whenever we see the start_of_record pattern, note patient and note numbers and start 
            # adding everything to the 'chunk' until we see the end_of_record.
            chunk = ''
            for line in text:
                record_start = re.findall(start_of_record_pattern,line,flags=re.IGNORECASE)
                if len(record_start):
                    patient, note = record_start[0]
                chunk += line

                # check to see if we have seen the end of one note
                record_end = re.findall(end_of_record_pattern, line,flags=re.IGNORECASE)

                if len(record_end):
                    # Now we have a full patient note stored in `chunk`, along with patient numerb and note number
                    # pass all to check_for_phone to find any phone numbers in note.
                    check_for_phone(patient,note,chunk.strip(), output_file)
                    # initialize the chunk for the next note to be read
                    chunk = ''

In [12]:
def deid_age(text_path= 'id.text', output_path = 'age.phi'):
    """
    TODO: edit for age instead of phone number
    Inputs: 
        - text_path: path to the file containing patient records
        - output_path: path to the output file.
    
    Outputs:
        for each patient note, the output file will start by a line declaring the note in the format of:
            Patient X Note Y
        then for each phone number found, it will have another line in the format of:
            start start end
        where the start is the start position of the detected phone number string, and end is the detected
        end position of the string both relative to the start of the patient note.
        If there is no phone number detected in the patient note, only the first line (Patient X Note Y) is printed
        to the output
    Screen Display:
        For each phone number detected, the following information will be displayed on the screen for debugging purposes 
        (these will not be written to the output file):
            start end phone_number
        where `start` is the start position of the detected phone number string, and `end` is the detected end position of the string
        both relative to the start of patient note.
    
    """
    # start of each note has the patter: START_OF_RECORD=PATIENT||||NOTE||||
    # where PATIENT is the patient number and NOTE is the note number.
    start_of_record_pattern = '^start_of_record=(\d+)\|\|\|\|(\d+)\|\|\|\|$'

    # end of each note has the patter: ||||END_OF_RECORD
    end_of_record_pattern = '\|\|\|\|END_OF_RECORD$'

    # open the output file just once to save time on the time intensive IO
    with open(output_path,'w+') as output_file:
        with open(text_path) as text:
            # initilize an empty chunk. Go through the input file line by line
            # whenever we see the start_of_record pattern, note patient and note numbers and start 
            # adding everything to the 'chunk' until we see the end_of_record.
            chunk = ''
            for line in text:
                record_start = re.findall(start_of_record_pattern,line,flags=re.IGNORECASE)
                if len(record_start):
                    patient, note = record_start[0]
                chunk += line

                # check to see if we have seen the end of one note
                record_end = re.findall(end_of_record_pattern, line,flags=re.IGNORECASE)

                if len(record_end):
                    # Now we have a full patient note stored in `chunk`, along with patient numerb and note number
                    
                    # check for age PHI
                    check_for_age(patient,note,chunk.strip(), output_file)
                    # initialize the chunk for the next note to be read
                    # TODO: Replace ages over 90 with 'Age over 90'
                    chunk = ''

In [13]:
deid_phone(text_path= '/home/sdabiri/BMI500/wk8/deid2021/python/id.text', 
           output_path = '/home/sdabiri/BMI500/wk8/deid2021/python/phone.phi')

8 1 552 564 201/324/1423
8 1 2296 2308 201-561-8910
8 4 987 999 201-223-4567
17 2 1197 1209 410-322-1419
17 2 1216 1228 603-960-5357
17 2 1235 1247 888-130-8121
17 2 1296 1308 450-928-6612
17 2 1338 1350 830-650-2352
17 2 1356 1368 301-680-6286
17 2 1376 1388 410-164-4517
28 2 1075 1087 285-482-5518
28 2 1104 1116 648-199-1703
41 2 2245 2257 204-943-1045
41 2 2288 2300 989-290-8303
44 4 1193 1205 135-442-9738
44 4 1694 1706 410-422-6213
47 2 431 443 301 944-5032
47 2 468 480 301 343-2822
48 3 961 973 410-555-9876
60 3 1628 1640 410 202-6694
60 3 1648 1660 410 671-9309
70 3 1729 1740 202 2671093
73 1 1766 1777 240444-1243
73 36 1890 1902 301 273 4516
82 2 1374 1386 410 392 0780
82 7 1109 1121 301-152-9058
96 1 1065 1076 202232-4455
139 1 1550 1562 858-492-5403
139 1 1567 1579 858-789-7896
139 1 1602 1614 415-999-8604


In [14]:
deid_age(text_path= '/home/sdabiri/BMI500/wk8/deid2021/python/id.text', 
           output_path = '/home/sdabiri/BMI500/wk8/deid2021/python/age.phi')

AttributeError: 'str' object has no attribute 'finditer'

In [1]:
import re

exampleString = '''
Jessica is 15 years old, and Daniel is 27 years old.
Edward is 97 years old, and his grandfather, Oscar, is 102. 
'''

In [2]:
ages = re.findall(r'\d{1,3}',exampleString)

print(ages)

['15', '27', '97', '102']


In [5]:
for i in range(1,len(ages)):
    if int(ages[i])> 90:
        ages[i] = 'age over 90'
        
print(ages)

['15', '27', 'age over 90', 'age over 90']
